In [ ]:
#| default_exp aurora

## Wrapper around an Aurora Postgres cluster

It implements a singleton pattern in order to prevent opening multiple database connections

In [ ]:
#| hide

import sys
from pathlib import Path

sys.path.append(str(Path('.').absolute().parent))

In [ ]:
#| exporti

from lakeinterface.config import ConfigManager
import json
import psycopg2
import boto3
import pandas as pd

In [ ]:
#| exporti

class Aurora(object):
    """
    A class to wrap connections to an AWS Aurora postgres cluster.
    Implemented as a singleton to reduce number of open connections
    ...

    Attributes
    ----------
    write_host : endpoint of writer instance of the cluster
    read_host : endpoint of reader instance of the cluster
    secret_id : id of AWS Secrets Manager secret containing connection credentials
    write_conn : pyscopg2 connection to write_host
    read_conn : pyscopg2 connection to read_host
    
    Methods
    -------
    __init__(config, profile='default'):
        Initializes the AWS Aurora connections using AWS profile_name and dict of parameters from ConfigManager
    
    aurora_connection(host):
        Core class for opening connections
    
    open_write_connection():
        Opens a pyscopg2 connection to write_host using aurora_connection method.

    open_read_connection():
        Opens a pyscopg2 connection to read_host using aurora_connection method.
        
    close_connections():
        Closes both read and write connections.
        
    query(sql):
        Executes a query on the read connection

    """
    _instance = None

    def __new__(cls, config, profile_name='default'):
        if cls._instance is None:
            print('Creating the object')
            cls._instance = super(Aurora, cls).__new__(cls)
            # Put any initialization here.
        return cls._instance
    
    def __init__(self, config, profile_name='default'):
        print('init-ing', self.__dict__)
        self.session = boto3.session.Session(profile_name=profile_name)
        
        self.write_host = config['aurora_writedb']
        self.read_host = config['aurora_readdb']
        self.secret_id = config['aurora_secret_id']
        
        self.open_read_connection()
        self.open_write_connection()
        
    def open_read_connection(self):
        self.read_conn = self.aurora_connection(
            self.read_host
        )
        
    def open_write_connection(self):
        self.write_conn = self.aurora_connection(
            self.write_host
        )
        
    def close_connections(self):
        print('Closing connections')
        self.write_conn.close()
        self.read_conn.close()
        
    def __exit__(self):
        self.close_connections()
        
    def aurora_connection(self, host):
        sman = self.session.client('secretsmanager')
        secret = sman.get_secret_value(SecretId=self.secret_id)
        credentials = json.loads(secret['SecretString'])

        return psycopg2.connect(
            database="bankdata",
            host=host,
            user=credentials['username'],
            password=credentials['password'],
            port="5432"
        )
    
    def query(self, sql):
        if self.read_conn.closed == 1:
            self.open_read_connection()
            
        with self.read_conn.cursor() as cursor:
            cursor.execute(sql)

            try:
                col_names = [desc[0] for desc in cursor.description]
                data = cursor.fetchall()
                self.read_conn.commit()
                return pd.DataFrame(data=data, columns=col_names)
            except Exception as e:
                self.read_conn.commit()
                raise e

In [ ]:
cfgmgr = ConfigManager(profile='personal')
cfg = cfgmgr.fetch_config('bankdata')

In [ ]:
db = Aurora(cfg)

Creating the object
init-ing {}


In [ ]:
sql = """
    SELECT 
        idrssd,
        filing_date,
        field_code,
        field_value
    FROM 
        bankdata.callreport_data
    WHERE
        field_code IN ('RCFD2170', 'RCON2170')
        AND idrssd in ('4114567', '2942690')
    ORDER BY filing_date
"""

df = db.query(sql)
df.head()

,idrssd,filing_date,field_code,field_value
0,2942690,2021-03-31,RCON2170,85382194.0
1,4114567,2021-03-31,RCFD2170,155797960.0
2,2942690,2021-06-30,RCON2170,97209737.0
3,4114567,2021-06-30,RCFD2170,161646734.0
4,2942690,2021-09-30,RCON2170,107850739.0
